# Global Food Prices

Understanding how xxxxx



___________________________________________
Author: Andreia Ferreira Almeida

In [1]:
# Imports
import os
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

### Loading data

In [2]:
proj_dir = os.path.dirname(os.path.abspath("wfp_market_food_prices.csv"))
data = os.path.join(proj_dir, "wfp_market_food_prices.csv")
print(data)

C:\reps\kaggel_tests\wfp_market_food_prices.csv


In [3]:
#Loading Data
gfp = pd.read_csv(data, engine='python')
gfp.head() # historico do preço p localidade p o produto durante 5 meses

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP


cm_id = (comodity_purchase) - id do produto


In [4]:
gfp.shape

(743914, 18)

### Data Cleaning

In [5]:
#renameming cols
gfp.columns = ['country_id', 'country_name', 'locality_id', 'locality_name', 'market_id', 'market_name', 'commodity_purchase_id', 'commodity_purchased', 'currency_id', 'currency', 'market_type_id', 'market_type', 'measurement_id', 'unit_of_goods_measurement', 'month_recorded', 'year_recorded','mp_price:_price_paid', 'source_supplying_price_information']
gfp.columns = [x.upper() for x in gfp.columns] #inputing_uppercase

In [6]:
#creating a list only with id cols
id_final = [col for col in gfp.columns if col.endswith('_ID')] #removing_id_cols#1
#droping id's col since doesnt add nothing to the analysis
gfp.drop(id_final, axis=1, inplace=True)#2
gfp.rename(columns={'MARKET_TYPE':'VENDOR_TYPE'}, inplace=True)
gfp.rename(columns={'MARKET_NAME':'MARKET_SHOPS'}, inplace=True)

In [7]:
gfp.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,COMMODITY_PURCHASED,CURRENCY,VENDOR_TYPE,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,1,2014,50.0,WFP
1,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,2,2014,50.0,WFP
2,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,3,2014,50.0,WFP
3,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,4,2014,50.0,WFP
4,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,5,2014,50.0,WFP


In [8]:
# create dataframe without year, month and price, to obtain one entry for each different shop
gfp_unique_shop = gfp[['COUNTRY_NAME','LOCALITY_NAME','MARKET_SHOPS','VENDOR_TYPE', 'COMMODITY_PURCHASED', 'CURRENCY']]
gfp_unique_shop.head()
initialSize = gfp_unique_shop.shape[0]
print("initial dataset size", initialSize)
gfp_unique_shop.drop_duplicates(inplace=True) # cair as colunas para poder analisar mercado único por país (mercados totais por pais)
gfp_unique_shop.reset_index(inplace=True, drop=True) #após drop_duplicates always do a reset
finalSize = gfp_unique_shop.shape[0]
print("final dataset size", finalSize)

initial dataset size 743914
final dataset size 16256


In [9]:
# renaming col in order to not misunderstand
gfp_unique_shop.rename(columns={'MARKET_NAME':'MARKET_SHOPS'}, inplace=True)

In [10]:
list_all_countries = gfp_unique_shop.groupby(by='COUNTRY_NAME')['COUNTRY_NAME'].nunique()
print( '[COUNTRIES] Total Contries in the df', len(list_all_countries))

[COUNTRIES] Total Contries in the df 74


In [11]:
list_all_markets = gfp_unique_shop.groupby(by='MARKET_SHOPS')['MARKET_SHOPS'].nunique()
print('[MARKETS] Total markets in the df', len(list_all_markets))

[MARKETS] Total markets in the df 1449


In [12]:
#TOTAL DE LOJAS EM QUE O NOME DA LOJA É IGUAL À LOCALIDADE
df_teste = gfp_unique_shop.loc[gfp_unique_shop['LOCALITY_NAME'] == gfp_unique_shop['MARKET_SHOPS']]

In [13]:
#df groupby object, para passar p df, utiliza-se a função: .apply(list)
df_teste_COUNTRY_2 = df_teste.groupby('COUNTRY_NAME').MARKET_SHOPS.apply(list)

In [14]:
df_teste_COUNTRY_2 = df_teste_COUNTRY_2.to_frame()

In [15]:
type(df_teste_COUNTRY_2)

pandas.core.frame.DataFrame

In [16]:
# listagem de todas as lojas diferentes em cada país
df_teste_COUNTRY_2['COUNT_SHOPS_C'] = df_teste_COUNTRY_2['MARKET_SHOPS'].str.len()

In [17]:
#tipos de comercio distinto
df_types_shop = gfp.VENDOR_TYPE.unique()

In [18]:
# type numpy.ndarray
print(df_types_shop)

['Retail' 'Wholesale' 'Producer' 'Farm Gate']


In [19]:
#passing numpy.ndarray to a df
df_unique_type_markets = pd.DataFrame(data=df_types_shop)    # values

In [20]:
#inputing names in the cols
df_unique_type_markets = df_unique_type_markets.rename(columns = {df_unique_type_markets.columns[0]:'VENDER_TYPE'})
#, df_unique_type_markets.columns[1]:'MARKET_TYPES'

In [21]:
df_unique_type_markets.head()

,VENDER_TYPE
0,Retail
1,Wholesale
2,Producer
3,Farm Gate


In [22]:
#mercados unicos por país
mercados_seleccionados = gfp[['COUNTRY_NAME', 'LOCALITY_NAME', 'MARKET_SHOPS']]
mercados_agrupados = mercados_seleccionados.groupby(['COUNTRY_NAME','MARKET_SHOPS']).nunique()
mercados_agrupados = mercados_agrupados.drop(['LOCALITY_NAME', 'MARKET_SHOPS', 'COUNTRY_NAME'], axis=1)
mercados_agrupados = mercados_agrupados.reset_index() #reseting index para poder usar a coluna country_name
mercados_unicos = mercados_agrupados.groupby('COUNTRY_NAME')['MARKET_SHOPS'].count()
mercados_unicos = mercados_unicos.to_frame()
mercados_unicos.head()

,MARKET_SHOPS
COUNTRY_NAME,
Afghanistan,11
Algeria,7
Armenia,6
Azerbaijan,1
Bangladesh,6


In [24]:
#nr total de produtos comercializados por mercado
df_mercados_dif_pais = gfp_unique_shop.copy()
df_mercados_dif_pais = df_mercados_dif_pais.groupby(['COUNTRY_NAME','MARKET_SHOPS']).size().reset_index()
df_mercados_dif_pais.rename(columns={0:'COUNT_COMMODITY'}, inplace=True) 
df_mercados_dif_pais = df_mercados_dif_pais.sort_values(by=['COUNT_COMMODITY'], ascending=False) # after a groupby reset_index
df_mercados_dif_pais = df_mercados_dif_pais.reset_index(drop=True) #do not drop the index in order to keep the countries
df_mercados_dif_pais.head()

,COUNTRY_NAME,MARKET_SHOPS,COUNT_COMMODITY
0,Colombia,Bogota,54
1,Rwanda,Byumba,50
2,Rwanda,Kibirizi,49
3,Rwanda,Gikongoro,49
4,Rwanda,Nkora,49


In [25]:
#média de produtos comercializados
print('média de produtos comercializados',df_mercados_dif_pais["COUNT_COMMODITY"].mean())

média de produtos comercializados 11.05098572399728


In [26]:
#média de produtos por mercado, por país
media_prod_pais = df_mercados_dif_pais.groupby('COUNTRY_NAME')["COUNT_COMMODITY"].mean()
media_prod_pais = media_prod_pais.sort_values(ascending=False).reset_index()
media_prod_pais.head() 

,COUNTRY_NAME,COUNT_COMMODITY
0,Turkey,44.000000
1,Rwanda,40.460317
2,State of Palestine,30.727273
3,Jordan,27.692308
4,Tajikistan,25.333333


In [27]:
#% commodity geral
df = gfp_unique_shop['COMMODITY_PURCHASED'].value_counts()
df

Maize                                            457
Rice (imported)                                  450
Sorghum                                          432
Rice (local)                                     419
Sugar                                            411
Rice                                             382
Wheat flour                                      367
Millet                                           362
Fuel (diesel)                                    337
Groundnuts (shelled)                             280
Eggs                                             271
Maize (white)                                    257
Tomatoes                                         256
Wheat                                            248
Fuel (petrol-gasoline)                           233
Meat (beef)                                      214
Oil (palm)                                       212
Onions                                           202
Beans (niebe)                                 

In [28]:
#vendor geral
df2 = gfp_unique_shop['VENDOR_TYPE'].value_counts()
df2

Retail       15311
Wholesale      918
Farm Gate       14
Producer        13
Name: VENDOR_TYPE, dtype: int64

In [29]:
#nr total de comercio global
df_com_global = gfp.groupby('VENDOR_TYPE').size()
df_com_global.head()

#GFP entrada de um produto repetida por vários meses

VENDOR_TYPE
Farm Gate       664
Producer        228
Retail       688878
Wholesale     54144
dtype: int64

In [30]:
#tipo de moedas
tipo_moeda = gfp_unique_shop.groupby('CURRENCY').size()
tipo_moeda.sort_values(ascending=False,inplace=True)
tipo_moeda = tipo_moeda.to_frame().reset_index().rename(columns={0:'COUNT_COIN'})
##questiono a validade

In [31]:
#tipo de produtos
tipo_produtos = gfp_unique_shop.groupby('COMMODITY_PURCHASED').size()
tipo_produtos.sort_values(ascending=False, inplace=True)
tipo_produtos.to_frame().reset_index().rename(columns={0:'COUNT_COIN'})
tipo_produtos.head()
##questiono a validade

COMMODITY_PURCHASED
Maize              457
Rice (imported)    450
Sorghum            432
Rice (local)       419
Sugar              411
dtype: int64

In [32]:
# TOP 5 alimentos por pais
a = gfp_unique_shop.groupby(['COUNTRY_NAME','COMMODITY_PURCHASED']).size()
a


COUNTRY_NAME  COMMODITY_PURCHASED                          
Afghanistan   Bread                                             8
              Exchange rate                                     8
              Fuel (diesel)                                     8
              Livestock (sheep, one-year-old alive female)      6
              Rice (low quality)                               10
              Wage (non-qualified labour, non-agricultural)     8
              Wage (qualified labour)                           8
              Wheat                                            11
Algeria       Apples                                            7
              Bananas                                           7
              Beans (white)                                     7
              Bread                                             6
              Carrots                                           7
              Cheese (dry)                                      7
              Eg

In [33]:
#out_put_analysis
#path_output = os.path.join(proj_dir, 'analysis.csv')
#df_mercados_pais.to_csv(path_output, sep='|', encoding='utf-8')

In [43]:
# TOP 5 alimentos por pais
f = gfp_unique_shop.groupby(['COUNTRY_NAME','COMMODITY_PURCHASED']) \
    .size() \
    .reset_index() \
    .rename(columns={0:'COUNT_MARKET_SELLING'}) \
    .sort_values(['COUNTRY_NAME', 'COUNT_MARKET_SELLING'], ascending=[True, False]) \
    .reset_index(drop=True)
f

,COUNTRY_NAME,COMMODITY_PURCHASED,COUNT_MARKET_SELLING
0,Afghanistan,Wheat,11
1,Afghanistan,Rice (low quality),10
2,Afghanistan,Bread,8
3,Afghanistan,Exchange rate,8
4,Afghanistan,Fuel (diesel),8
5,Afghanistan,"Wage (non-qualified labour, non-agricultural)",8
6,Afghanistan,Wage (qualified labour),8
7,Afghanistan,"Livestock (sheep, one-year-old alive female)",6
8,Algeria,Apples,7
9,Algeria,Bananas,7


In [48]:
aaa = gfp.loc[gfp['COUNTRY_NAME'] == 'Colombia']
aaa.shape

(8296, 11)

In [52]:
bbb = gfp_unique_shop.loc[gfp_unique_shop['COUNTRY_NAME'] == 'Afghanistan']
bbb.shape

(67, 6)

In [53]:
ccc = gfp_unique_shop.loc[gfp_unique_shop['COUNTRY_NAME'] == 'Colombia']
ccc.shape

(169, 6)